This notebook applies different methodologies to improve the ranking of ligands beyond the scores directly obtained from docking scores.

The methods here applied are reviewed by  Arcineaga and Lange (2014).

- Ligand Efficiency
- Ranking Positions

In [2]:
import pandas as pd
import numpy as np
import glob, os, sys
sys.path.append(r'..')

These resultas are considered as 'DockScore', and corresponds to the best ligand ligand score given by the tool employed.

In [3]:
data_dir = '../data'
# Resultados CSAR
df_ad4_results_LE = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_AD4_LE.csv', index_col=0)
df_ad4_results_LC = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_AD4_LC.csv', index_col=0)
df_vina_results   = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_VINA.csv', index_col=0)
df_vinardo_results = pd.read_csv(F'{data_dir}/vs_docking_crys_ensemble_VINARDO.csv', index_col=0)
# Resultados DUD
patho_to_json_vrd8_file = glob.glob((os.path.join('..', 'data', 
                       'vs_docking_DUD2006_vs_402_crys_vinardo_8x.csv')))[0]
df_vinardo_DUD = pd.read_csv(patho_to_json_vrd8_file, index_col=0)

## Ligand Efficiency

Herein Ligand Efficiency corresponds to the quotient between the best ligand's score and the number of heavy atoms in the ligand.

Because we only need the number  of heavy atoms, we can directly load the molecule using `rdkit`

In [120]:
def get_ligand_efficiency(scores_df, n_atoms_df):
    # First we need to make sure the index are the same
    assert all(scores_df.index == n_atoms_df.index), 'Index are not the same'
    assert 'ActiveInactive' in scores_df.columns, 'Make sure "ActiveInactive" column exists in scores_df'
    # Then the division is performed
    lef_df = scores_df.drop(['ActiveInactive'], axis = 1).div(n_atoms_df.Num_heavy_atoms, axis = 0)
    # Finally we reinsert the ActiveInactive Column at the begining
    lef_df.insert(0, 'ActiveInactive', scores_df['ActiveInactive'])
    return lef_df

def 

In [27]:
from rdkit import Chem, RDLogger
from rdkit.Chem import rdchem
# neccesary to ignore warning realte to molecule kekulization
RDLogger.DisableLog('rdApp.*')

In [67]:
files_csar_ligs_path = os.path.join(*'../../ARCHIVOS/CRISTALES/LIGS_CDK2/CSAR/sdf/*'.split('/'))
files_csar_ligs = glob.glob(files_csar_ligs_path)
# alphanumerical ordering
files_csar_ligs.sort(key= lambda x: int(x.split('/')[-1].split('.')[0].replace('CS', '')))
# We now load each molecule in a dictionary
csar_natm_ligs_dic = {
    file.split('/')[-1].split('.')[0] : Chem.SDMolSupplier(file, sanitize = False)[0].GetNumHeavyAtoms()
    for file in files_csar_ligs
}

In [69]:
nAtoms_csar_ligs = pd.DataFrame(csar_natm_ligs_dic, index = ['Num_heavy_atoms'])
# Now transpose and arder the dataframe
nAtoms_csar_ligs = (nAtoms_csar_ligs.T)

In [111]:
# Now we divide each ligand's score between the number of heavy atoms:
